<a href="https://colab.research.google.com/github/monteroserra/elderly-companion-ai/blob/main/notebooks/elderly_companion_ai_notebook_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## elderly-companion-ai notebook 2: text and voice models


In [4]:
!pip install transformers SpeechRecognition gtts pydub --quiet


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Checking for Rust toolchain....
      
      Cargo, the Rust package manager, is not installed or is not on PATH.
      This package requires Rust and Cargo to compile extensions. Install it through
      the system's package manager or via https://rustup.rs/
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import speech_recognition as sr
from gtts import gTTS
from pydub import AudioSegment
from pydub.playback import play

# Record Audio Function
def record_audio(filename="recorded_audio.wav"):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Recording... Speak now!")
        audio = recognizer.listen(source)
        print("Recording complete.")
    
    # Save the audio as a WAV file
    with open(filename, "wb") as f:
        f.write(audio.get_wav_data())
    print(f"Audio saved as {filename}")
    return filename

# Playback Audio Function
def play_audio(filename):
    try:
        # Load the audio file
        audio = AudioSegment.from_file(filename)
        print(f"Playing audio: {filename}")
        play(audio)
    except Exception as e:
        print(f"Error playing audio: {e}")

# Execute main functionality
if __name__ == "__main__":
    audio_file = record_audio()  # Record and save audio
    play_audio(audio_file)       # Play back the saved audio


In [2]:
import os
import speech_recognition as sr
from gtts import gTTS
from transformers import pipeline

# Initialize speech recognizer
recognizer = sr.Recognizer()

# Define LLM (using OpenAI's GPT-2 model for this example)
generator = pipeline("text-generation", model="gpt2")

# Function to convert speech to text
def speech_to_text(audio_file):
    with sr.AudioFile(audio_file) as source:
        print("Processing audio for transcription...")
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            print(f"Recognized text: {text}")
            return text
        except sr.UnknownValueError:
            print("Speech Recognition could not understand the audio")
            return None
        except sr.RequestError as e:
            print(f"Could not request results from Speech Recognition service; {e}")
            return None

# Function to generate response using an LLM
def generate_response(input_text):
    print("Generating response with LLM...")
    response = generator(input_text, max_length=50, num_return_sequences=1)
    generated_text = response[0]['generated_text']
    print(f"Generated text: {generated_text}")
    return generated_text

# Function to convert text to speech
def text_to_speech(text, output_file="response.mp3"):
    print("Converting text to speech...")
    tts = gTTS(text=text, lang='en')
    tts.save(output_file)
    os.system(f"mpg123 {output_file}")  # Play the generated audio file



ModuleNotFoundError: No module named 'speech_recognition'

## Record voice

In [ ]:

!apt-get install -y portaudio19-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 0s (1,000 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 123634 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-

In [ ]:
!pip install sounddevice scipy


In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write
import os

def record_voice(filename="output.wav", duration=5, sample_rate=44100):
    """
    Records voice from the microphone and saves it as a .wav file.

    :param filename: Name of the output file.
    :param duration: Duration of recording in seconds.
    :param sample_rate: Sampling rate in Hz (default is 44100 Hz).
    """
    print(f"Recording for {duration} seconds...")
    try:
        # Record audio
        audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype="int16")
        sd.wait()  # Wait until recording is finished

        # Save as a WAV file
        write(filename, sample_rate, audio)
        print(f"Recording saved as {filename}")
    except Exception as e:
        print(f"An error occurred while recording: {e}")



In [ ]:
# Example usage
record_filename = "test_audio.wav"  # Change the name if needed
record_duration = 5  # Duration in seconds

record_voice(filename=record_filename, duration=record_duration)

# Verify the file is saved
if os.path.exists(record_filename):
    print(f"File {record_filename} is ready for testing.")
else:
    print("Recording failed.")

Recording for 5 seconds...
An error occurred while recording: Error querying device -1
Recording failed.


In [ ]:
import sounddevice as sd

print(sd.query_devices())

In [ ]:
# Example usage
# Record or provide an audio file here (use a pre-recorded .wav file for demo purposes)
audio_file_path = "path_to_audio.wav"  # Replace with your .wav file path

# Step 1: Speech-to-Text
input_text = speech_to_text(audio_file_path)

if input_text:
    # Step 2: Generate Response
    output_text = generate_response(input_text)

    # Step 3: Text-to-Speech
    text_to_speech(output_text)
else:
    print("No valid input detected.")